In [1]:
import numpy as np
import h5py
import trimesh
import matplotlib.pyplot as plt
from random import random, uniform
import dlib
from scipy.optimize import lsq_linear
import pyglet
import time

In [2]:
%run features/basel_features/index_loader.ipynb
%run basel_model/BaselModel.ipynb
%run optimizers/GradientDescent.ipynb
%run optimizers/initializers/EstimationRefinementInitializer.ipynb
%run optimizers/inputs/SingleImageInput.ipynb
%run optimizers/inputs/ContinuousImagesInput.ipynb
%run optimizers/outputs/FaceModelMeshViewer.ipynb

Using TensorFlow backend.


In [3]:
baselModel = BaselModel('./model2017-1_bfm_nomouth.h5')
        
# get indexes of model feature points and associated dlib feature point numbers, indexed at 1
feature_point_indexes, dlib_points = get_feature_point_indexes('./feature_points.csv')
        
# extract only the feature points
shape_mean_features = baselModel.getShapeMean(pointIndexes=feature_point_indexes)
shape_pcaBasis_features = baselModel.getShapePCABasis(pointIndexes=feature_point_indexes)
        
triangles = baselModel.getModelMeshTriangles()
    
initializer = EstimationRefinementInitializer(numIterations=1)

optimizer = GradientDescent(
        mean = shape_mean_features,
        pcaBasis = shape_pcaBasis_features,
        pcaVariance = baselModel.getShapePCAVariance(),
        numIterations = 5000,
        initializer = initializer)

inputHandler = ContinuousImagesInput(
        ['./test_images/test_image_1.jpeg', './test_images/test_image_2.jpeg'],
        './shape_predictor_68_face_landmarks.dat',
        dlib_points)

sceneGenerator = SceneGenerator(
        baselModel.getShapeMean(),
        baselModel.getShapePCABasis(),
        baselModel.getShapePCAVariance(),
        triangles)

sceneViewer = SceneViewer(sceneGenerator.getScene(np.zeros((299, 1)), 1, 1, 1), start_loop=False)

def benchmark(dt):
    print("TESTING", flush=True)
    a, r, t, s = optimizer.optimize(inputHandler.getNext())
    scene = sceneGenerator.getScene(a, r, t, s)
    scene.set_camera()
    sceneViewer.scene = scene
    sceneViewer.on_draw()
        
pyglet.clock.schedule(benchmark)

#a, r, t, s = optimizer.optimize(inputHandler.getNext())

#outputHandler.handle(a, r, t, s)
pyglet.app.run()
            

TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
TESTING
